In [ ]:
!pip3 install pandas numpy datasets huggingface_hub --upgrade

In [ ]:
import pandas as pd
import numpy as np
import os
import json

BASE_PATH = "/home/sun/Downloads/tg_challenge/"
# from google.colab import drive
# drive.mount('/content/drive')

# Data

In [ ]:
PATH_TO_JSON_FILE = os.path.join(BASE_PATH, 'language_identification_dataset_100_with_messages_and_regexps.json')

with open(PATH_TO_JSON_FILE, 'r') as f:
    data = json.load(f)

In [ ]:
list(data.keys())

In [ ]:
new_data = dict()
for key in data:
    if len(data[key]) == 1:
        new_data[key] = [data[key][0]]
    else:
        new_data[key] = [data[key]]

In [ ]:
len(new_data['TGLANG_LANGUAGE_BASIC'][0][1])
# new_data.keys()

In [ ]:
# data_for_df = [{"class": key, "text": value[0][i]} for key, value in new_data.items() for i in range(len(value[0]))]
data_for_df = []

for key, value in new_data.items():
    for i in range(len(value[0])):
        if key == 'TGLANG_LANGUAGE_BASIC':
            for j in range(len(value[0][i])):
                data_for_df.append({'class': key, 'text': value[0][i][j]})
        else:
            data_for_df.append({'class': key, 'text': value[0][i]})

In [ ]:
data_for_modeling = pd.DataFrame(data_for_df).sample(frac=1)

# Load data

In [ ]:
modeling_data_file = os.path.join(BASE_PATH, "data_for_modeling.json")
data_for_modeling.to_json(modeling_data_file)
data = pd.read_json(modeling_data_file)

In [ ]:
data['class'].value_counts()

In [ ]:
IMBALANCED_CLASSES = [
    "TGLANG_LANGUAGE_REGEX",
    "TGLANG_LANGUAGE_OTHER",
]

balanced_based = data[~data['class'].isin(IMBALANCED_CLASSES)]

In [ ]:
balanced_dfs = [data[data['class'] == CLASS].sample(100, random_state=42) for CLASS in IMBALANCED_CLASSES]

In [ ]:
balanced_data = pd.concat([balanced_based, *balanced_dfs], axis=0)

In [ ]:
balanced_data.head()

In [ ]:
# data[data['class'] == 'TGLANG_LANGUAGE_CPLUSPLUS'].text.str.contains("//").astype(int)

# all(isinstance(value, str) for value in data['text'])

# Modeling

In [ ]:
!pip3 install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, PredefinedSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
### Splitting into folds
from typing import List
from tqdm import tqdm

K_FOLDS = 5

In [ ]:



# evaluation utils
def generate_folding(k_folds: int, df: pd.DataFrame) -> List[int]:

    len_folding = df.shape[0] // k_folds
    all_folding = []
    for i in range(k_folds):

        if i != k_folds - 1:
            all_folding += [i + 1 for _ in range(len_folding)]
        else:
            len_folds_computed = df.shape[0] - len(all_folding)
            all_folding += [i + 1 for _ in range(len_folds_computed)]
    return all_folding




In [ ]:
def get_test_fold_for_iteration(fold_indices, test_fold_num):
    return [0 if f == test_fold_num else -1 for f in fold_indices]

In [ ]:
### Refactor
def train_multiple_folds(n_folds: int, df: pd.DataFrame, vectorizer: object) -> List[float]:

    X = vectorizer.fit_transform(df['text'])
    y = df['class']

    test_folding = generate_folding(k_folds=n_folds, df=df)

    clf = MultinomialNB()
    all_scores = []

    for fold_num in tqdm(range(n_folds)):  # As we have 4 folds
        current_test_fold = get_test_fold_for_iteration(test_folding, fold_num)
        ps = PredefinedSplit(current_test_fold)
        scores = cross_val_score(clf, X, y, cv=ps, scoring='f1_macro')
        all_scores.extend(scores)

    return all_scores

In [ ]:
data['text'].head()

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False)

In [ ]:
X = vectorizer.fit_transform(data['text'])
y = data['class']

In [ ]:
X.shape, len(data), len(y)

In [ ]:
vectorizer.get_params()

In [ ]:
vectorizer.idf_

In [ ]:
from sklearn.metrics import f1_score

clf = MultinomialNB()
# usual traning would be here
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)


In [ ]:
mnb = clf.fit(train_X, train_y)
mnb

In [ ]:
mnb.feature_count_.shape

In [ ]:
mnb.feature_log_prob_.shape

In [ ]:
mnb.alpha

In [ ]:
mnb.score(val_X, val_y)

In [ ]:
from skl2onnx import to_onnx
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType
to_onnx?

In [ ]:
cols = X.shape[1]
cols

In [ ]:
initial_types = [('float_input', FloatTensorType([None, cols]))]

In [ ]:
onnx_clf = to_onnx(clf, initial_types=initial_types)

In [ ]:
onnx_path = os.path.join(BASE_PATH, "mnb-1.onnx")
with open(onnx_path, "wb") as f:
    f.write(onnx_clf.SerializeToString())

In [ ]:
!ls -lah mnb-1.onnx

In [ ]:
pred_y = clf.predict(val_X)

In [ ]:
pred_y[:20]

In [ ]:
val_y[:20]

In [ ]:
!pip3 install onnxruntime

In [ ]:
import onnxruntime as rt

In [ ]:
sess = rt.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

In [ ]:
input_name

In [ ]:
label_name

In [ ]:
t_val_X = val_X[:1000].toarray().astype('float32')

In [ ]:
t_val_X.shape, t_val_X.dtype

In [ ]:
pred_onx = sess.run([label_name], {input_name: t_val_X})[0]

In [ ]:
pred_onx[:20]

In [ ]:
equals = pred_onx[:1000] == pred_y[:1000]
sum(equals) == 1000

In [ ]:
# test_folding = generate_folding(k_folds=K_FOLDS, df=data)

# # k-fold cross validation
# all_scores = []

# for fold_num in tqdm(range(K_FOLDS)):
#     current_test_fold = get_test_fold_for_iteration(test_folding, fold_num + 1)
#     ps = PredefinedSplit(current_test_fold)
#     # breakpoint()
#     # print(pd.Series(current_test_fold).value_counts())
#     scores = cross_val_score(clf, X, y, cv=ps)
#     all_scores.extend(scores)

# print(all_scores)

In [ ]:
!pip install scikit-learn onnx onnxruntime skl2onnx

### TODO: per-class evaluation

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


# X = vectorizer.fit_transform(balanced_data['text'])
# y = balanced_data['class']

# train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)
# clf = MultinomialNB()
# clf.fit(train_X, train_y)

# pred_y = clf.predict(val_X)

# # precision_per_class = precision_score(val_y, pred_y, average=None)
# # recall_per_class = recall_score(val_y, pred_y, average=None)
# # f1_per_class = f1_score(val_y, pred_y, average=None)

# report = classification_report(val_y, pred_y, output_dict=True)



In [ ]:
# report_df = pd.DataFrame(report).T.reset_index().rename(columns={'index': 'class'})
# report_df

# report_df[report_df['class'] == '']
# data['class'].value_counts().sort_values()

In [ ]:
# report_df[report_df['class'] == 'TGLANG_LANGUAGE_REGEX'].reset_index()


In [ ]:
# for i, row in data.iterrows():
#     if type(row['text']) == list:
#         print(i)

# data.loc[900, 'text']